## 0. Install and Import Dependencies

In [1]:
!pip install ibm_watson

In [2]:
exist = !which ffmpeg
if not exist:
  !curl https://johnvansickle.com/ffmpeg/releases/ffmpeg-release-amd64-static.tar.xz -o ffmpeg.tar.xz \
     && tar -xf ffmpeg.tar.xz && rm ffmpeg.tar.xz
  ffmdir = !find . -iname ffmpeg-*-static
  path = %env PATH
  path = path + ':' + ffmdir[0]
  %env PATH $path
print('')
if exist:
	print('Done!')


Done!


In [3]:
import subprocess
from ibm_watson import SpeechToTextV1
from ibm_watson.websocket import RecognizeCallback, AudioSource
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

ModuleNotFoundError: No module named 'ibm_watson'


# 1. Extract Audio

In [ ]:
command = 'ffmpeg -i content/video/a.mp4 -ab 160k -ar 44100 -vn content/audio/audio.wav'
subprocess.call(command, shell=True)

# 2. Setup STT Service

In [ ]:
apikey = '4IwdLMj6aBGv_lK9yPnMz8gHBvWOMnF0xzt4xZF_V_2i'
url = 'https://api.au-syd.speech-to-text.watson.cloud.ibm.com/instances/d1585919-bd55-424e-975e-3e75a3ac56b5'

In [ ]:
# Setup service
authenticator = IAMAuthenticator(apikey)
stt = SpeechToTextV1(authenticator=authenticator)
stt.set_service_url(url)

## 3. Open Audio Source and Convert

In [ ]:
with open('/content/audio/audio.wav', 'rb') as f:
    res = stt.recognize(audio=f, content_type='audio/wav', model='en-AU_NarrowbandModel').get_result()

In [ ]:
res

# 4. Process Results and Output to Text

In [ ]:
len(res['results'])

In [ ]:
text = [result['alternatives'][0]['transcript'].rstrip() + '.\n' for result in res['results']]

In [ ]:
text = [para[0].title() + para[1:] for para in text]
transcript = ''.join(text)
with open('output.txt', 'w') as out:
    out.writelines(transcript)

In [ ]:
transcript

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [ ]:
# Download NLTK resources (if not already downloaded)
nltk.download('punkt')
nltk.download('stopwords')

In [ ]:

# Tokenization
tokens = word_tokenize(transcript.lower())  # Convert to lowercase for consistency
print(tokens)
tokens = [token for token in tokens if token.isalpha()]  # Remove non-alphabetic tokens
print(tokens)


In [ ]:

# Remove stop words
stop_words = set(stopwords.words('english'))
filtered_tokens = [token for token in tokens if token not in stop_words]


In [ ]:

# Data Visualization - Word Cloud
filtered_text = ' '.join(filtered_tokens)
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(filtered_text)


In [ ]:

# Display the Word Cloud using matplotlib
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud - After Preprocessing')
plt.show()


In [ ]:

# Data Visualization - Word Frequency Bar Chart
word_freq = nltk.FreqDist(filtered_tokens)
common_words = word_freq.most_common(10)  # Display top 10 words
print(common_words)


In [ ]:

# Plotting the bar chart
plt.figure(figsize=(12, 6))
plt.bar(*zip(*common_words))
plt.xlabel('Words')
plt.ylabel('Frequency')
plt.title('Top 10 Word Frequencies - After Preprocessing')
plt.xticks(rotation=45, ha='right')
plt.show()